# Copernicus: query and ingest

https://documentation.dataspace.copernicus.eu/APIs/STAC.html

https://documentation.dataspace.copernicus.eu/APIs/S3.html#example-script-to-download-product-using-boto3

In [1]:
import os
os.environ["abcli_path_bash"] = "{}/git/awesome-bash-cli/bash".format(os.getenv("HOME"))

In [2]:
import boto3
from pystac_client import Client
from tqdm import tqdm
from abcli import fullname
from abcli import file, path, string
from abcli.modules import objects
from abcli.plugins.metadata import get_from_object
from blue_geo import env
from blue_geo import NAME, VERSION
from blue_geo.catalog.copernicus import CopernicusSentinel2Datacube
from blue_geo.catalog import get_datacube
from blue_geo.logger import logger

logger.info(f"{NAME}-{VERSION}.{fullname()}, built on {string.pretty_date()}")

🌐  blue_geo-4.257.1.abcli-9.245.1, built on 21 August 2024, 20:40:02


In [3]:
URL = "https://catalogue.dataspace.copernicus.eu/stac"
client = Client.open(URL)

---

## query

In [4]:
object_name = f"query-{string.timestamp()}"
logger.info(f"📂 {object_name}")

🌐  📂 query-2024-08-21-20-40-03-53378


In [5]:
assert CopernicusSentinel2Datacube.query(
    object_name=object_name,
    bbox=[-122.78 - 0.1, 51.83 - 0.1, -122.78 + 0.1, 51.83 + 0.1],
    datetime="2024-07-30/2024-08-09",
    count=-1,
)

🌐  🔎 CopernicusSentinel2Datacube.query -> query-2024-08-21-20-40-03-53378
🌐  🔎 collections: ['SENTINEL-2']
🌐  🔎 bbox: [-122.88, 51.73, -122.68, 51.93]
🌐  🔎 datetime: 2024-07-30/2024-08-09
🌐  16 datacubes(s) found.
🌐  🧊 01: datacube-copernicus-sentinel_2-S2A_MSIL1C_20240731T191911_N0511_R099_T10UDC_20240801T003519-SAFE
🌐  🧊 02: datacube-copernicus-sentinel_2-S2A_MSIL1C_20240731T191911_N0511_R099_T10UEC_20240801T003519-SAFE
🌐  🧊 03: datacube-copernicus-sentinel_2-S2A_MSIL1C_20240807T190911_N0511_R056_T10UDC_20240808T002811-SAFE
🌐  🧊 04: datacube-copernicus-sentinel_2-S2A_MSIL1C_20240807T190911_N0511_R056_T10UEC_20240808T002811-SAFE
🌐  🧊 05: datacube-copernicus-sentinel_2-S2A_MSIL2A_20240731T191911_N0511_R099_T10UDC_20240801T023153-SAFE
🌐  🧊 06: datacube-copernicus-sentinel_2-S2A_MSIL2A_20240731T191911_N0511_R099_T10UEC_20240801T023153-SAFE
🌐  🧊 07: datacube-copernicus-sentinel_2-S2A_MSIL2A_20240807T190911_N0511_R056_T10UDC_20240808T020449-SAFE
🌐  🧊 08: datacube-copernicus-sentinel_2-S2A_

In [6]:
datacube_id = get_from_object(object_name, "datacube_id")[0]
logger.info(f"🧊 {datacube_id}")

🌐  🧊 datacube-copernicus-sentinel_2-S2A_MSIL1C_20240731T191911_N0511_R099_T10UDC_20240801T003519-SAFE


## ingest

In [7]:
datacube = get_datacube(datacube_id)

In [8]:
assert datacube.update_metadata(verbose=True)
item = datacube.metadata["Item"]

🌐  blue_geo.catalog.generic.generic.CopernicusSentinel2Datacube@copernicus[datacube-copernicus-sentinel_2-S2A_MSIL1C_20240731T191911_N0511_R099_T10UDC_20240801T003519-SAFE]:.update_metadata()
🌐  ids: ['S2A_MSIL1C_20240731T191911_N0511_R099_T10UDC_20240801T003519.SAFE']
🌐  🧊 S2A_MSIL1C_20240731T191911_N0511_R099_T10UDC_20240801T003519.SAFE: 2024-07-31 19:19:11.024000+00:00 @ QUICKLOOK, PRODUCT
🌐  href: /eodata/Sentinel-2/MSI/L1C/2024/07/31/S2A_MSIL1C_20240731T191911_N0511_R099_T10UDC_20240801T003519.SAFE
🌐  bucket_name: eodata
🌐  s3_prefix: Sentinel-2/MSI/L1C/2024/07/31/S2A_MSIL1C_20240731T191911_N0511_R099_T10UDC_20240801T003519.SAFE
🌐  66 file(s).
🌐  #1: DATASTRIP/DS_2APS_20240801T003519_S20240731T192211/MTD_DS.xml
🌐  #2: DATASTRIP/DS_2APS_20240801T003519_S20240731T192211/QI_DATA/FORMAT_CORRECTNESS.xml
🌐  #3: DATASTRIP/DS_2APS_20240801T003519_S20240731T192211/QI_DATA/GENERAL_QUALITY.xml
🌐  #4: DATASTRIP/DS_2APS_20240801T003519_S20240731T192211/QI_DATA/GEOMETRIC_QUALITY.xml
🌐  #5: DATA

In [9]:
success, metadata = datacube.ingest(datacube.datacube_id)
assert success
logger.info(metadata)

🌐  blue_geo.catalog.generic.generic.CopernicusSentinel2Datacube@copernicus[datacube-copernicus-sentinel_2-S2A_MSIL1C_20240731T191911_N0511_R099_T10UDC_20240801T003519-SAFE]: -> datacube-copernicus-sentinel_2-S2A_MSIL1C_20240731T191911_N0511_R099_T10UDC_20240801T003519-SAFE
🌐  🪄
🌐  None


In [10]:
success, bucket, s3_prefix = datacube.get_bucket(verbose=True)

🌐  href: /eodata/Sentinel-2/MSI/L1C/2024/07/31/S2A_MSIL1C_20240731T191911_N0511_R099_T10UDC_20240801T003519.SAFE
🌐  bucket_name: eodata
🌐  s3_prefix: Sentinel-2/MSI/L1C/2024/07/31/S2A_MSIL1C_20240731T191911_N0511_R099_T10UDC_20240801T003519.SAFE


In [11]:
list_of_items = bucket.objects.filter(Prefix=s3_prefix)
extension = "TCI.jp2"
dryrun = False

datacube_path = objects.object_path(datacube_id)


error_count = 0
for item in tqdm(list_of_items):
    item_suffix = item.key.split(f"{s3_prefix}/",1)[1]
    if not item_suffix:
        continue
    
    item_filename = os.path.join(datacube_path,item_suffix)
    if not path.create(file.path(item_filename)):
        error_count += 1
        continue
    if item_filename.endswith(os.sep):
        continue
    
    if not item_filename.endswith(extension):
        continue
    
    logger.info("{} -> {}".format(item.key, item_filename))
    if dryrun:
        continue

    try:
        bucket.download_file(item.key, item_filename)
    except Exception as e:
        logger.error(e)
        error_count += 1
        continue

if error_count:
    logger.error(f"{error_count} error(s).")

1it [00:00,  1.09it/s]🌐  Sentinel-2/MSI/L1C/2024/07/31/S2A_MSIL1C_20240731T191911_N0511_R099_T10UDC_20240801T003519.SAFE/GRANULE/L1C_T10UDC_A047572_20240731T192211/IMG_DATA/T10UDC_20240731T191911_TCI.jp2 -> /Users/kamangir/storage/abcli/datacube-copernicus-sentinel_2-S2A_MSIL1C_20240731T191911_N0511_R099_T10UDC_20240801T003519-SAFE/GRANULE/L1C_T10UDC_A047572_20240731T192211/IMG_DATA/T10UDC_20240731T191911_TCI.jp2
66it [00:10,  6.29it/s]


---

In [12]:
# END